## Performing Batch Cofolding Structure Predictions

This section provides an example of submitting a batch request to predict the cofolding of up to five molecules with a common receptor protein. The workflow applies to both Chai1 and Boltz2 models, each supporting their respective constraints and user-defined settings.

Before running this notebook, please ensure you:

1. Are logged in by running `dm login EMAIL` in the terminal
2. Have a Token registered and saved on the file


In [ ]:
!pip install deepmirror

In [ ]:
!dm login <YOUREMAIL>

In [ ]:
import time

import deepmirror.api as api

### Define some helper functions

In [ ]:
def submit_cofold(chains: list[dict], user_settings: dict) -> str:
    response = api.structure_prediction(chains, user_settings)
    return response["task_id"]


def batch_cofold_submission(
    receptor_chains: dict,
    ligand_id: str,
    ligand_smiles: list[str],
    user_settings: dict,
) -> list[str] | None:
    if len(ligand_smiles) > 5:
        print(
            "Please provide less than 5 ligands for batch structure prediction"
        )
        return None
    if len(ligand_smiles) == 0:
        print(
            "Please provide at least one ligand for batch structure prediction"
        )
        return None

    receptor_chainIDs = [chain["label"] for chain in receptor_chains]
    if ligand_id in receptor_chainIDs:
        print(
            "Ligand ID cannot be the same as other chains in receptor. Please provide a unique ID for ligand"
        )
        return None

    task_ids = []
    for ligand in ligand_smiles:
        chains = receptor_chains + [
            {
                "label": ligand_id,
                "value": ligand,
                "type": "ligand",
            }
        ]
        task_ids.append(submit_cofold(chains, user_settings))
    return task_ids


def check_status(task_ids: list[str]) -> None:
    is_finished = [False] * len(task_ids)
    while not all(is_finished):
        for idx, task_id in enumerate(task_ids):
            response = api.get_structure_prediction(task_id)
            status = response["status"]
            if status == "completed":
                is_finished[idx] = True
                print(f"Task ID: {task_id} - Current status: {status}")
            else:
                print(
                    f"Task ID: {task_id} - Current status: {status} - Waiting 2 min for completion..."
                )
        time.sleep(120)


def download_results(task_ids: list[str]) -> None:
    for task_id in task_ids:
        with open(f"result-{task_id}.zip", "wb") as f:
            f.write(api.download_structure_prediction(task_id))

### Describe the receptor Protein (may include protein chains, RNA/DNA and Ions)

In [ ]:
receptor_chains = [
    {
        "label": "A",
        "value": "MTEYKLVVVGADGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVIDGETCLLDILDTAGQEEYSAMRDQYMRTGEGFLCVFAINNTKSFEDIHHYREQIKRVKDSEDVPMVLVGNKCDLPSRTVDTKQAQDLARSYGIPFIETSAKTRQGVDDAFYTLVREIRKHKEK",
        "type": "protein",
    }
]

### Add constraints -- Chai or Boltz

In [ ]:
constraint_1 = {
    "chainA": "A",
    "res_idxA": "F28",
    "chainB": "A",
    "res_idxB": "Q99",
    "connection_type": "contact",
    "confidence": 1.0,
    "min_distance_angstrom": 3.0,
    "max_distance_angstrom": 5.0,
    "comment": "Inter Protein Chain Contact",
    "restraint_id": "restraint_0",
}

constraint_2 = {
    "chainA": "B",
    "res_idxA": "",
    "chainB": "A",
    "res_idxB": "D57",
    "connection_type": "pocket",
    "confidence": 1.0,
    "min_distance_angstrom": 4,
    "max_distance_angstrom": 5,
    "comment": "Protein - Small Molecule Pocket Interaction",
    "restraint_id": "restraint_1",
}

constraints = [constraint_1, constraint_2]
user_settings = {"constraints": constraints, "model": "chai"}

### List all the molecules to cofold using their smiles (less than 5)

In [ ]:
ligand_smiles = [
    "C1N(C2OC(COP(OP(O)(O)=O)(O)=O)C(O)C2O)C2N=C(N)NC(=O)C=2N=1",
    "C1N(C2OC(CC3N(OC)CCC3)C(O)C2O)C2N=C(N)NC(=O)C=2N=1",
    "C1N(C2OCCC2OC)C2C=CC=CC=2N=1",
]

### Submit their structure prediction requests

In [ ]:
submission_task_ids = batch_cofold_submission(
    receptor_chains, "B", ligand_smiles, user_settings
)

In [ ]:
print("Task IDs of submitted requests: ", submission_task_ids)

### Check Status of submissions and download results as zipped files

In [ ]:
check_status(submission_task_ids)

download_results(submission_task_ids)